In [ ]:
# Copyright (c) 2020 ZZH

In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, AveragePooling2D, MaxPool2D
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
import os
import numpy as np
import math

In [ ]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
epochs = 100
lr = 0.1
batch_size = 128
REGULARIZER  = 0.0001
checkpoint_save_path =  './Model/DRN92/'
log_dir = os.path.join("Model","DRN92_logs")

In [ ]:
#数据导入及数据增强
cifar10 = tf.keras.datasets.cifar10
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)
mean = [125.307, 122.95, 113.865]  #np.mean()
std = [62.9932, 62.0887, 66.7048]  #np.std()
for i in range(3):
    x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
    x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]

DataGenTrain = tf.keras.preprocessing.image.ImageDataGenerator(
               rotation_range = 15,
               width_shift_range = 0.125,
               height_shift_range = 0.125,
               horizontal_flip = True,
               vertical_flip = False,
               shear_range=0.125,
               zoom_range = 0.125)
DataGenTrain.fit(x_train)

In [ ]:
def scheduler(epoch):  #HTD(-6,3) with WarmingUp
    start = -6.0
    end = 3.0
    if epoch < 5:
        return 0.02 * epoch + 0.02
    return lr / 2.0 * (1- math.tanh( (end-start)*epoch/epochs + start))

In [ ]:
class BNRelu(Model):
    def __init__(self):
        super(BNRelu,self).__init__()
        self.bn = BatchNormalization(momentum=0.9)
        self.relu = Activation('relu')
    def call(self,inputs):
        x = self.bn(inputs)
        outputs = self.relu(x)
        return outputs

In [ ]:
class ConvBNRelu(Model):
    def __init__(self,channels,kernel_size,strides,padding):
        super(ConvBNRelu,self).__init__()
        self.c = Conv2D(filters=channels, kernel_size=kernel_size,strides=strides, padding=padding,use_bias=False,
                                kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
        self.b = BNRelu()                                                 
    def call(self,inputs):
        x = self.c(inputs)
        outputs = self.b(x)
        return outputs

In [ ]:
class GroupConvBNRelu(Model):
    def __init__(self,channels,kernel_size,cardinality,strides):
        super(GroupConvBNRelu,self).__init__()
        self.group = []
        self.models = []
        self.cardinality = cardinality
        self.slice = channels//cardinality
        for i in range(self.cardinality):
            self.models.append(Conv2D(filters=self.slice, kernel_size=kernel_size, strides=strides, padding='same', use_bias=False,
                               kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER)))
        self.b = BNRelu()   
    def call(self,inputs):
        self.group = []
        for i in range(self.cardinality):
            x = inputs[:,:,:, i*self.slice : (i+1)*self.slice]
            self.group.append(self.models[i](x))
        y = tf.concat(self.group,axis = -1)
        outputs = self.b(y)        
        return outputs

In [ ]:
class DualPathBlock(Model):
    def __init__(self,channels_top,channels_bottom,k,strides,increase):
        super(DualPathBlock,self).__init__()
        self.increase = increase
        self.channels_bottom = channels_bottom
        self.b1 = BNRelu()
        self.c1 = ConvBNRelu(channels=channels_top, kernel_size=1, strides=1, padding='same')
        self.c2 = GroupConvBNRelu(channels=channels_top, kernel_size=3, cardinality=32, strides=strides)
        self.c3 = Conv2D(filters=channels_bottom + k, kernel_size=1, strides=1, padding='same',
                         kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
        if self.increase:
            self.c4 = Conv2D(filters=channels_bottom + k, kernel_size=1, strides=strides, padding='same', 
                             kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
    def call(self,inputs):
        if isinstance(inputs, list):
            res_path = inputs[0]
            dense_path = inputs[1]
            inputs = tf.concat(inputs, axis=-1)
        x = self.b1(inputs)
        x = self.c1(x)
        x = self.c2(x)
        x = self.c3(x)
        y1 = x[:,:,:,:self.channels_bottom]
        y2 = x[:,:,:,self.channels_bottom:]
        if self.increase:
            proj = self.c4(inputs)
            res_path = proj[:,:,:,:self.channels_bottom]
            dense_path = proj[:,:,:,self.channels_bottom:]
        res_path = res_path + y1
        dense_path = tf.concat([dense_path, y2], axis=-1)
        return [res_path,dense_path]

In [ ]:
class DRN92(Model):
    def __init__(self,block_list):
        super(DRN92,self).__init__()
        self.channels_top = 96
        self.channels_bottom = 256
        self.increment = [16,32,24,128]
        self.c1 = ConvBNRelu(channels=32, kernel_size=3, strides=1, padding='same')
        self.blocks = []
        self.layer_num = 0
        for i in range(len(block_list)):
            if i == 0:
                self.blocks.append(DualPathBlock(channels_top=self.channels_top,channels_bottom=self.channels_bottom,
                                                 k=self.increment[i],strides=1,increase=True))
            else:
                self.blocks.append(DualPathBlock(channels_top=self.channels_top,channels_bottom=self.channels_bottom,
                                                 k=self.increment[i],strides=2,increase=True))
            self.layer_num += 1
            for _ in range(1,block_list[i]):
                self.blocks.append(DualPathBlock(channels_top=self.channels_top,channels_bottom=self.channels_bottom,
                                                 k=self.increment[i],strides=1,increase=False))
                self.layer_num += 1
            self.channels_top *= 2
            self.channels_bottom *= 2
        self.b1 = BNRelu()
        self.p1 = GlobalMaxPool2D()
        self.p2 = GlobalAveragePooling2D()
        self.f1 = Dense(10,activation='softmax',kernel_initializer="he_normal",
                        kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
    def call(self,inputs):
        x = self.c1(inputs)
        for i in range(self.layer_num):
            x = self.blocks[i](x)
        x = tf.concat(x, axis= -1)
        x = self.b1(x)
        x1 = self.p1(x)
        x2 = self.p2(x)
        x = 0.5 * (x1 + x2)
        y = self.f1(x)
        return y

In [ ]:
model = DRN92([3,4,20,3])

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True, clipnorm=2.),
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
              metrics=['accuracy'])

callbacks = [
            tf.keras.callbacks.LearningRateScheduler(scheduler),  #学习率衰减表
            #tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, min_lr=0.0001, patience=10, cooldown=0)
            tf.keras.callbacks.ModelCheckpoint(     #模型保存
                filepath = checkpoint_save_path,
                save_weights_only = False,
                monitor = 'val_accuracy',
                save_best_only = True),
#             tf.keras.callbacks.EarlyStopping(       #早停
#                 monitor = 'val_accuracy',
#                 patience=15, 
#                 baseline=None),
            tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=False)  #保存计算图
]

hist = model.fit(DataGenTrain.flow(x_train,y_train,batch_size=batch_size,shuffle=True),
                 epochs=epochs,
                 validation_data=(x_test,y_test),
                 validation_freq=1,
                 callbacks=callbacks)

model.summary()

In [ ]:
#结果可视化
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use({'figure.figsize':(6,4)})

plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_loss'], label='val_loss')
plt.legend()
plt.show()
plt.plot(hist.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
#tensorboard可视化
#!tensorboard --logdir=./Model/DRN92_logs
#http://localhost:6006/

In [ ]:
print('best result: {:.2f}%  ({}epochs)'.format(100*max(hist.history['val_accuracy']),1+hist.history['val_accuracy'].index(max(hist.history['val_accuracy']))))
# best result: 95.78%  (96epoch)